In [1]:
%matplotlib widget

import torch
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


import hybrid_mpfd_solver as hybrid
import torch_mpfd_solver as solver

from training_data import generateData
import pandas as pd
from torch.utils.data import Dataset, DataLoader


import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
### Generate data and save it to the .csv ###

#generateData(10)

In [3]:
df = pd.read_csv('high_fidelity_training_data.csv')

df['tN'] = df['tN'].tolist()
df['psiInitial'] = df['psiInitial'].apply(eval).tolist()
df['psiFinal'] = df['psiFinal'].apply(eval).tolist()

In [4]:
data = list(df.itertuples(index=False, name=None))
dataset = hybrid.CustomDataset(data, hybrid.downsamplefun, 10)


batch_size = 1
def custom_collate_fn(batch):
    tN_batch = torch.tensor([item[0] for item in batch], dtype=torch.float64)
    psiInitial_batch = [torch.tensor(item[1], dtype=torch.float64) for item in batch]
    psiFinal_batch = [torch.tensor(item[2], dtype=torch.float64) for item in batch]
    return tN_batch, psiInitial_batch, psiFinal_batch


dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=custom_collate_fn)

In [5]:

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)

for batch in dataloader:
    tN_batch, psiInitial_batch, psiFinal_batch = batch
    print("tN_batch:", tN_batch)
    print("psiInitial_batch:", psiInitial_batch)
    print("psiFinal_batch:", psiFinal_batch)




tN_batch: tensor([471.2120], dtype=torch.float64)
psiInitial_batch: [tensor([-104.0377, -104.0166, -103.9917, -103.9683, -103.9434, -103.9278,
        -103.9048, -103.8865, -103.8488, -103.8327, -103.8163, -103.8001,
        -103.7704, -103.7461, -103.7255, -103.6946, -103.6614, -103.6540,
        -103.6184, -103.5830, -103.5541, -103.5296, -103.4867, -103.4614,
        -103.4345, -103.4072, -103.3840, -103.3479, -103.3316, -103.3045,
        -103.2788, -103.2443, -103.2250, -103.1950, -103.1740, -103.1413,
        -103.1205, -103.1035, -103.0851, -103.0684,  -19.5036],
       dtype=torch.float64)]
psiFinal_batch: [tensor([-104.0377, -104.0117, -103.9867, -103.9625, -103.9389, -103.9156,
        -103.8926, -103.8697, -103.8466, -103.8233, -103.7990, -103.7719,
        -103.7355, -103.6658, -103.4808, -102.9085, -101.1306,  -96.1894,
         -85.5928,  -70.2304,  -55.0607,  -43.6636,  -36.3252,  -31.7571,
         -28.7904,  -26.7456,  -25.2612,  -24.1390,  -23.2639,  -22.5652,
       

In [6]:
# Instantiate the convolutional neural network and the combined model

nn_model = hybrid.NN()
hybrid_model = hybrid.HybridModel(solver.ModelRun, nn_model)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(hybrid_model.parameters(), lr=0.001)


# Parameters for the coarser model
dt = 120
zN = 40

In [9]:
for batch in dataloader:
    tN_batch, psiInitial_batch, psiFinal_batch = batch
    print(solver.setup(120,tN_batch[0],40,psiInitial_batch[0]))
    #hybrid_model(dt,tN_batch,zN,psiFinal_batch)




(tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
        29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.]), tensor([   0.,  120.,  240.,  360.,  480.,  600.,  720.,  840.,  960., 1080.,
        1200., 1320., 1440.]), 1.0, 39, 13, 40, [tensor([-129.6480, -129.6125, -129.5835, -129.5550, -129.5332, -129.5079,
        -129.4868, -129.4684, -129.4489, -129.4270, -129.4057, -129.3813,
        -129.3565, -129.3382, -129.3135, -129.2775, -129.2386, -129.2030,
        -129.1847, -129.1436, -129.1138, -129.0944, -129.0823, -129.0490,
        -129.0221, -129.0064, -128.9828, -128.9639, -128.9368, -128.9119,
        -128.8890, -128.8735, -128.8492, -128.8297, -128.8089, -128.7912,
        -128.7733, -128.7336, -128.7098], requires_grad=True)], tensor([-129.6748]), tensor([-45.5527]), {'thetaR': 0.075, 'thetaS': 0.287, 'alpha': 1611000.0, 'beta': 3.96, 'A': 1175000.0, 'gamma': 4.74, 'K

c:\Users\Valde\OneDrive\Dokumenter\GitHub\PINNs\torch_mpfd_solver.py:180: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psi = torch.tensor(psiInitial[1:-1], dtype=torch.float32)


In [8]:
'''
num_epochs = 10



for epoch in range(num_epochs):
    for tN_batch, psiInitial_batch, psiFinal_batch in dataloader:
        tN_batch = tN_batch.float()
        psiInitial_batch = psiInitial_batch.float()
        psiFinal_batch = psiFinal_batch.float()

        optimizer.zero_grad()
        outputs = hybrid_model(tN_batch, psiInitial_batch)
        loss = criterion(outputs, psiFinal_batch)
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

'''

"\nnum_epochs = 10\n\n\n\nfor epoch in range(num_epochs):\n    for tN_batch, psiInitial_batch, psiFinal_batch in dataloader:\n        tN_batch = tN_batch.float()\n        psiInitial_batch = psiInitial_batch.float()\n        psiFinal_batch = psiFinal_batch.float()\n\n        optimizer.zero_grad()\n        outputs = hybrid_model(tN_batch, psiInitial_batch)\n        loss = criterion(outputs, psiFinal_batch)\n        loss.backward()\n        optimizer.step()\n\n        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')\n\n"